## Inserção base Completa

In [8]:
import pandas as pd
import psycopg2
import math

In [9]:
amostraP = pd.read_csv('../csv/amostraBHPadronizadaTotal.csv')
amostraP.head()

,Unnamed: 0,number,CEP,street,bairro,city,state
0,0,277,31060390.0,RUA OTAVIO BARRETO,Bairro Boa Vista,BELO HORIZONTE,MG
1,1,1025,30640060.0,RUA BARAO DE COROMANDEL,Bairro do Barreiro,BELO HORIZONTE,MG
2,2,623,30620180.0,RUA PONTA GROSSA,Bairro Araguaia,BELO HORIZONTE,MG
3,3,148,30622060.0,RUA BUENO DE RIVERA,Bairro Araguaia,BELO HORIZONTE,MG
4,4,326,30620730.0,RUA TAPAIARAS,Bairro Araguaia,BELO HORIZONTE,MG


In [14]:
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [11]:
def get_max_request_id(cur):
    
    query = """ SELECT MAX(request_id)
                FROM "Request"
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior[0][0]

In [12]:
amostraP = amostraP.fillna( value={'number':'REMOVER','street':	'REMOVER','city':'REMOVER', 'CEP':''})

In [13]:
batch_size = 50000  # Tamanho do lote

num_batches = math.ceil(len(amostraP) / batch_size)

request_id = get_max_request_id(cur_crawler)

for i in range(num_batches):
    
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(amostraP))

    numero = amostraP['number'].iloc[start_idx:end_idx].to_list()
    street = amostraP['street'].iloc[start_idx:end_idx].to_list()
    city = amostraP['city'].iloc[start_idx:end_idx].to_list()
    district = amostraP['city'].iloc[start_idx:end_idx].to_list()
    postcode = amostraP['CEP'].iloc[start_idx:end_idx].to_list()

    query_insercao = """
        INSERT INTO "Request" (
            request_id,
            full_address,
            number,
            public_place,
            district,
            city,
            zip_code,
            state
        )
        VALUES """
    for i in range(0, len(numero)):
        if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
            continue
        else:
            if postcode[i] != '':
                cod_postal = int(postcode[i])
            else:
                cod_postal = postcode[i]
            request_id += 1
            full_address = street[i].replace("'","''") + ', ' + str( numero[i])+ ', ' + city[i].replace("'","''") + ', ' + 'MG'
            query_insercao += """(
                {},
                '{}',
                '{}',
                '{}',
                '{}',
                '{}',
                '{}',
                '{}'),\n""".format(request_id, full_address, 
                                    numero[i], street[i].replace("'","''"),
                                    '',
                                    city[i].replace("'","''"),
                                    cod_postal, 'MG')
        

    query_insercao =query_insercao[:-2] +';'
    #print(query_insercao)

    cur_crawler.execute(query_insercao)
    conn_crawler.commit()

cur_crawler.close()


In [15]:
# Testando para ver se deu certo

def get_max_adress(cur):
    
    query = """ SELECT *
                FROM "Request"
                ORDER BY request_id DESC
                LIMIT 1
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior

maior = get_max_adress(cur_crawler)
print(maior)

[(3700828, 'AVENIDA OLEGARIO MACIEL, 2247, BELO HORIZONTE, MG', 'AVENIDA OLEGARIO MACIEL', '', 'BELO HORIZONTE', 'MG', '30180118', None, '2247')]


## Inserção Amostra

In [1]:
import pandas as pd 

In [3]:
amostra = pd.read_csv('..\csv\amostraBH5mil.csv')
#amostra = pd.read_csv('../csv/baseBHtransformada.csv')
amostra.head()

,Unnamed: 0,IDEND,ID_EDC,ID_LOGRADOURO,SIGLA_TIPO_LOGRADOURO,DESC_TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,LETRA_IMOVEL,ID_BAIRRO_POPULAR,...,ID_BAIRRO_OFICIAL,NUM_BAIRRO_OFICIAL,TIPO_BAIRRO_OFICIAL,NOME_BAIRRO_OFICIAL,ID_REGIONAL,NOME_REGIONAL,CEP,GEOMETRIA,LATITUDE,LONGITUDE
0,0,05058100277,443458,50581,RUA,RUA,OTAVIO BARRETO,277,NaN,15,...,242,830,Bairro,Boa Vista,3,LESTE,31060390.0,"(-19.89688040126723, -43.9038375951244)",-19.896880,-43.903838
1,1,00789101025,451710,7891,RUA,RUA,BARAO DE COROMANDEL,1025,NaN,481,...,221,200,Bairro,do Barreiro,1,BARREIRO,30640060.0,"(-19.976545248550647, -44.024423385549426)",-19.976545,-44.024423
2,2,01490000623,453157,14900,RUA,RUA,PONTA GROSSA,623,NaN,105,...,247,203,Bairro,Araguaia,1,BARREIRO,30620180.0,"(-19.983122702414487, -43.99861503074346)",-19.983123,-43.998615
3,3,09214600148,466439,92146,RUA,RUA,BUENO DE RIVERA,148,NaN,220,...,247,203,Bairro,Araguaia,1,BARREIRO,30622060.0,"(-19.983744735197362, -43.9911336510454)",-19.983745,-43.991134
4,4,06665100326,461913,66651,RUA,RUA,TAPAIARAS,326,NaN,221,...,247,203,Bairro,Araguaia,1,BARREIRO,30620730.0,"(-19.984679564865772, -43.99589383410702)",-19.984680,-43.995894


In [4]:
amostra.columns

Index(['Unnamed: 0', 'IDEND', 'ID_EDC', 'ID_LOGRADOURO',
       'SIGLA_TIPO_LOGRADOURO', 'DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO',
       'NUMERO_IMOVEL', 'LETRA_IMOVEL', 'ID_BAIRRO_POPULAR',
       'NUM_BAIRRO_POPULAR', 'NOME_BAIRRO_POPULAR', 'ID_BAIRRO_OFICIAL',
       'NUM_BAIRRO_OFICIAL', 'TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL',
       'ID_REGIONAL', 'NOME_REGIONAL', 'CEP', 'GEOMETRIA', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [5]:
amostraP = amostra[['DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL', 'CEP']].copy()

amostraP['street'] = amostra['DESC_TIPO_LOGRADOURO'] + ' ' + amostra['NOME_LOGRADOURO']

amostraP.drop(['DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO'], axis=1, inplace=True)

amostraP['bairro'] = amostra['TIPO_BAIRRO_OFICIAL'] + ' ' + amostra['NOME_BAIRRO_OFICIAL']

amostraP.drop(['TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL'], axis=1, inplace=True)



amostraP.head()


,NUMERO_IMOVEL,CEP,street,bairro
0,277,31060390.0,RUA OTAVIO BARRETO,Bairro Boa Vista
1,1025,30640060.0,RUA BARAO DE COROMANDEL,Bairro do Barreiro
2,623,30620180.0,RUA PONTA GROSSA,Bairro Araguaia
3,148,30622060.0,RUA BUENO DE RIVERA,Bairro Araguaia
4,326,30620730.0,RUA TAPAIARAS,Bairro Araguaia


In [6]:
amostraP['city'] = 'BELO HORIZONTE'
amostraP['state'] = 'MG'

In [7]:
amostraP.rename(columns={'NUMERO_IMOVEL':'number'}, inplace=True)

In [8]:
amostraP

,number,CEP,street,bairro,city,state
0,277,31060390.0,RUA OTAVIO BARRETO,Bairro Boa Vista,BELO HORIZONTE,MG
1,1025,30640060.0,RUA BARAO DE COROMANDEL,Bairro do Barreiro,BELO HORIZONTE,MG
2,623,30620180.0,RUA PONTA GROSSA,Bairro Araguaia,BELO HORIZONTE,MG
3,148,30622060.0,RUA BUENO DE RIVERA,Bairro Araguaia,BELO HORIZONTE,MG
4,326,30620730.0,RUA TAPAIARAS,Bairro Araguaia,BELO HORIZONTE,MG
...,...,...,...,...,...,...
763224,70,31050750.0,RUA PINDARE,Bairro Mariano de Abreu,BELO HORIZONTE,MG
763225,315,31870750.0,RUA DOM SILVERIO GOMES PIMENTA,Bairro Belmonte,BELO HORIZONTE,MG
763226,275,31870750.0,RUA DOM SILVERIO GOMES PIMENTA,Bairro Belmonte,BELO HORIZONTE,MG
763227,2255,30180118.0,AVENIDA OLEGARIO MACIEL,Seção Urbana Décima,BELO HORIZONTE,MG


In [10]:
amostraP.to_csv('../csv/amostraBHPadronizadaTotal.csv')

### Inserção no Banco

In [8]:
import pandas as pd

In [9]:
amostraP = pd.read_csv('..\csv\amostraBHPadronizada5mil.csv')

In [10]:
amostraP = amostraP[['number', 'CEP', 'street', 'bairro', 'city', 'state']]

In [11]:
amostraP

,number,CEP,street,bairro,city,state
0,138,31370710.0,RUA TRES,Bairro do Trevo,BELO HORIZONTE,MG
1,1156,30855500.0,RODOVIA PRESIDENTE JUSCELINO KUBITSCHEK,Bairro do Pilar,BELO HORIZONTE,MG
2,759,30350532.0,RUA ABEL ARAUJO,Bairro Santa Lúcia,BELO HORIZONTE,MG
3,178,31710140.0,RUA CARMO DO PARANAIBA,Bairro Itapoã,BELO HORIZONTE,MG
4,138,31920740.0,AVENIDA NOSSA SENHORA DA SAUDE,Bairro Pirajá,BELO HORIZONTE,MG
...,...,...,...,...,...,...
4995,175,31360300.0,RUA URCA,Bairro Itatiaia,BELO HORIZONTE,MG
4996,10,31270802.0,AVENIDA OTACILIO NEGRAO DE LIMA,Bairro São Luíz,BELO HORIZONTE,MG
4997,853,30690510.0,RUA MANJERICAO,Bairro Lindéia,BELO HORIZONTE,MG
4998,160,30535310.0,ALAMEDA SAPOTI,Bairro Dom Cabral,BELO HORIZONTE,MG


In [11]:
import psycopg2
import io

In [12]:
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [13]:
def get_max_request_id(cur):
    
    query = """ SELECT MAX(request_id)
                FROM "Request"
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior[0][0]

In [14]:
amostraP = amostraP.fillna( value={'number':'REMOVER','street':	'REMOVER','city':'REMOVER', 'CEP':''})

In [15]:
amostraP.to_csv('../csv/amostraBHPadronizadaTotal.csv')

In [16]:
amostraP.columns

Index(['number', 'CEP', 'street', 'bairro', 'city', 'state'], dtype='object')

In [17]:
numero = amostraP['number'].to_list()
street = amostraP['street'].to_list()
city = amostraP['city'].to_list()
district = amostraP['city'].to_list()
postcode = amostraP['CEP'].to_list()
bairro = amostraP['bairro'].to_list()

In [18]:
request_id = get_max_request_id(cur_crawler)

In [19]:
request_id

2937599

In [20]:
query_insercao = """
    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES """
for i in range(0, len(numero)):
    if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
        continue
    else:
        if postcode[i] != '':
            cod_postal = int(postcode[i])
        else:
            cod_postal = postcode[i]
        request_id += 1
        full_address = street[i].replace("'","''") + ', ' + str( numero[i])+ ', ' + city[i].replace("'","''") + ', ' + 'MG'
        query_insercao += """(
            {},
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}'),\n""".format(request_id, full_address, 
                                numero[i], street[i].replace("'","''"),
                                '',
                                city[i].replace("'","''"),
                                cod_postal, 'MG')
    

query_insercao =query_insercao[:-2] +';'

KeyboardInterrupt: 

In [ ]:
print(query_insercao)


    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES (
            2852600,
            'RUA TRES, 138, BELO HORIZONTE, MG',
            '138',
            'RUA TRES',
            '',
            'BELO HORIZONTE',
            '31370710',
            'MG'),
(
            2852601,
            'RODOVIA PRESIDENTE JUSCELINO KUBITSCHEK, 1156, BELO HORIZONTE, MG',
            '1156',
            'RODOVIA PRESIDENTE JUSCELINO KUBITSCHEK',
            '',
            'BELO HORIZONTE',
            '30855500',
            'MG'),
(
            2852602,
            'RUA ABEL ARAUJO, 759, BELO HORIZONTE, MG',
            '759',
            'RUA ABEL ARAUJO',
            '',
            'BELO HORIZONTE',
            '30350532',
            'MG'),
(
            2852603,
            'RUA CARMO DO PARANAIBA, 178, BELO HORIZONTE, MG',
            '178',
            'RU

In [29]:
cur_crawler.execute(query_insercao)
cur_crawler.close()
conn_crawler.commit()

In [30]:
request_id

2857599

In [33]:
request_id = get_max_request_id(cur_crawler)
request_id

2857599

In [20]:

def get_max_adress(cur):
    
    query = """ SELECT *
                FROM "Request"
                ORDER BY request_id DESC
                LIMIT 1
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior

#query = 'SELECT * FROM Request  AS r ORDER BY r.request_id DESC LIMIT 1'


In [37]:
ultimo = get_max_request_id(cur_crawler)

In [38]:
ultimo

[(2857599,
  'RUA MANJERICAO, 211, BELO HORIZONTE, MG',
  'RUA MANJERICAO',
  '',
  'BELO HORIZONTE',
  'MG',
  '30690510',
  None,
  '211')]

## Experimentos diversos 

### Obs: Modifiquei esse código inúmeras vezes, o de cima é mais confiável

In [21]:
#import pandas as pd
#import psycopg2

request_id = get_max_request_id(cur_crawler)
query_insercao = """
    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES """
for i in range(0, len(numero)):
    if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
        continue
    else:
        if postcode[i] != '':
            cod_postal = int(postcode[i])
        else:
            cod_postal = postcode[i]
        request_id += 1
        full_address = street[i].replace("'","''") +  ', ' + str( numero[i]) + ', ' + bairro[i].replace("'","''") + ', ' + city[i].replace("'","''") + ', ' + 'MG'
        query_insercao += """(
            {},
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}'),\n""".format(request_id, full_address, 
                                numero[i], street[i].replace("'","''"),
                                '',
                                city[i].replace("'","''"),
                                cod_postal, 'MG')
    

query_insercao =query_insercao[:-2] +';'

print(query_insercao)

#bairro[i].replace("'","''")


    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES (
            2907600,
            'RUA TRES, 138, Bairro do Trevo, BELO HORIZONTE, MG',
            '138',
            'RUA TRES',
            '',
            'BELO HORIZONTE',
            '31370710',
            'MG'),
(
            2907601,
            'RODOVIA PRESIDENTE JUSCELINO KUBITSCHEK, 1156, Bairro do Pilar, BELO HORIZONTE, MG',
            '1156',
            'RODOVIA PRESIDENTE JUSCELINO KUBITSCHEK',
            '',
            'BELO HORIZONTE',
            '30855500',
            'MG'),
(
            2907602,
            'RUA ABEL ARAUJO, 759, Bairro Santa Lúcia, BELO HORIZONTE, MG',
            '759',
            'RUA ABEL ARAUJO',
            '',
            'BELO HORIZONTE',
            '30350532',
            'MG'),
(
            2907603,
            'RUA CARMO DO PARANAIBA, 178, B

In [2]:
import pandas as pd
import psycopg2
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [22]:
cur_crawler.execute(query_insercao)
ultimo = get_max_adress(cur_crawler)
print(ultimo)
cur_crawler.close()
conn_crawler.commit()

[(2912599, 'RUA MANJERICAO, 211, Bairro Lindéia, BELO HORIZONTE, MG', 'RUA MANJERICAO', '', 'BELO HORIZONTE', 'MG', '30690510', None, '211')]


In [23]:
ultimo = get_max_adress(cur_crawler)

InterfaceError: cursor already closed

In [25]:
ultimo

[(2912599,
  'RUA MANJERICAO, 211, Bairro Lindéia, BELO HORIZONTE, MG',
  'RUA MANJERICAO',
  '',
  'BELO HORIZONTE',
  'MG',
  '30690510',
  None,
  '211')]